# Course: ICS 574
# Prject: Alchemists Project - EDA
- Date: May 1st 2024
- Project Members
    - AHMED DHAFER ALQARNI, ID: 201453160
    - WALEED ABDULLAH ALFAIFI, ID: 201640920
    - ALGHAMDI, BANDAR, ID: 202206560

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import seaborn as sns
import numpy as np


In [2]:
# Create a Spark session
spark = SparkSession.builder.appName("Decision Tree Classifier Example 4").getOrCreate()




24/03/31 07:31:50 WARN Utils: Your hostname, Bandars-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.100.154 instead (on interface en0)
24/03/31 07:31:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/31 07:31:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [13]:
# Load the datasets as pandas DataFrames
stocks_df = pd.read_csv("./datasets/Tadawul_stcks.csv")
brent_df = pd.read_csv("./datasets/BrentOilPrices.csv")
gold_df = pd.read_csv("./datasets/Gold_Daily.csv")

# Preprocess the stock data
stocks_specific_df = stocks_df[(stocks_df['sectoer'] == 'Materials')]
stocks_specific_df.rename(columns={'date': 'Date'}, inplace=True)
stocks_specific_df.rename(columns={'close': 'Stock_Price'}, inplace=True)
stocks_specific_df['Date'] = pd.to_datetime(stocks_specific_df['Date'])
brent_df['Date'] = pd.to_datetime(brent_df['Date'])
gold_df['Date'] = pd.to_datetime(gold_df['Date'])
brent_df.rename(columns={'Price': 'Brent_Price'}, inplace=True)
gold_df.rename(columns={'Price': 'Gold_Price'}, inplace=True)
stocks_specific_df.rename(columns={'sectoer': 'Sector'}, inplace=True)
brent_df = brent_df[['Date', 'Brent_Price']]
gold_df = gold_df[['Date', 'Gold_Price']]
stocks_specific_df.fillna(method='ffill', inplace=True)
stocks_brent_df = pd.merge(stocks_specific_df, brent_df, on='Date', how='inner')
stocks_brent_gold_df = pd.merge(stocks_brent_df, gold_df, on='Date', how='inner')




/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_16247/1058981075.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_specific_df.rename(columns={'date': 'Date'}, inplace=True)
/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_16247/1058981075.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_specific_df.rename(columns={'close': 'Stock_Price'}, inplace=True)
/var/folders/8r/wc11r95d3lgcc57lr4h_9m0w0000gn/T/ipykernel_16247/1058981075.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [14]:
# Create a Spark DataFrame
stocks_brent_gold_df_spark = spark.createDataFrame(stocks_brent_gold_df)

In [15]:
# print columns of the DataFrame
stocks_brent_gold_df_spark.columns

['symbol',
 'name',
 'trading_name ',
 'Sector',
 'Date',
 'open',
 'high',
 'low',
 'Stock_Price',
 'change',
 'perc_Change',
 'volume_traded ',
 'value_traded',
 'no_trades ',
 'Brent_Price',
 'Gold_Price']

In [16]:
# because we are going to use the DecisionTreeRegressor, we need to convert the categorical data to numerical data
indexer = StringIndexer(inputCol="Sector", outputCol="SectorIndex")
indexed = indexer.fit(stocks_brent_gold_df_spark).transform(stocks_brent_gold_df_spark)


In [17]:
# index the name of the stock
indexer = StringIndexer(inputCol="name", outputCol="StockIndex")
indexed = indexer.fit(indexed).transform(indexed)

In [18]:
indexed.show()

+------+--------------------+---------------+---------+-------------------+-----+-----+-----+-----------+------+-----------+--------------+-------------+----------+-----------+----------+-----------+----------+
|symbol|                name|  trading_name |   Sector|               Date| open| high|  low|Stock_Price|change|perc_Change|volume_traded | value_traded|no_trades |Brent_Price|Gold_Price|SectorIndex|StockIndex|
+------+--------------------+---------------+---------+-------------------+-----+-----+-----+-----------+------+-----------+--------------+-------------+----------+-----------+----------+-----------+----------+
|  1201|Takween Advanced ...|        TAKWEEN|Materials|2020-03-05 00:00:00| 7.68| 7.68|  7.5|       7.52| -0.04|      -0.53|      183425.0|    1386959.3|     275.0|      51.29|    1690.5|        0.0|      33.0|
|  1202|Middle East Paper...|          MEPCO|Materials|2020-03-05 00:00:00|13.66| 13.8|13.54|      13.54| -0.02|      -0.15|      584937.0|   7975687.54|   

In [25]:
# create a feature vector by combining the features that we are going to use
vectorAssembler = VectorAssembler(inputCols=["Stock_Price", "volume_traded ", "value_traded", "no_trades ", "Brent_Price", "Gold_Price"], outputCol="features")

In [26]:
assembledData = vectorAssembler.transform(indexed)

In [27]:
# Create a VectorAssembler for the features
stocks_brent_gold_df_spark = assembledData

In [29]:
stocks_brent_gold_df_spark.show()

+------+--------------------+---------------+---------+-------------------+-----+-----+-----+-----------+------+-----------+--------------+-------------+----------+-----------+----------+-----------+----------+--------------------+
|symbol|                name|  trading_name |   Sector|               Date| open| high|  low|Stock_Price|change|perc_Change|volume_traded | value_traded|no_trades |Brent_Price|Gold_Price|SectorIndex|StockIndex|            features|
+------+--------------------+---------------+---------+-------------------+-----+-----+-----+-----------+------+-----------+--------------+-------------+----------+-----------+----------+-----------+----------+--------------------+
|  1201|Takween Advanced ...|        TAKWEEN|Materials|2020-03-05 00:00:00| 7.68| 7.68|  7.5|       7.52| -0.04|      -0.53|      183425.0|    1386959.3|     275.0|      51.29|    1690.5|        0.0|      33.0|[7.52,183425.0,13...|
|  1202|Middle East Paper...|          MEPCO|Materials|2020-03-05 00:00:

In [35]:
# Define the DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol="features", labelCol="StockIndex")

In [36]:
# define the evaluator
# note the metricName parameter is rmse, which stands for Root Mean Squared Error. This is the default metric for regression problems.
# Other metrics include r2 (R squared) and mae (Mean Absolute Error)
# see the documentation for more details: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="StockIndex", predictionCol="prediction", metricName="rmse")

In [37]:
# Define the grid of hyperparameters
# We will use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for dt.maxDepth and 5 values for dt.maxBins, this grid will have 3 x 5 = 15 parameter settings for CrossValidator to choose from.
# see the documentation for more details: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [2, 3, 4, 5, 6, 7, 8, 9, 10]) \
    .addGrid(dt.maxBins, [10, 20, 40, 80, 100]) \
    .build()

In [38]:
# Define the CrossValidator
# We will use a CrossValidator to select the best model.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# see the documentation for more details: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [39]:
# fit the cv model with the assembled data
cvModel = cv.fit(stocks_brent_gold_df_spark)

24/03/31 07:58:34 WARN CacheManager: Asked to cache already cached data.
24/03/31 07:58:34 WARN CacheManager: Asked to cache already cached data.


In [40]:
# get the average cross-validated RMSE
# note the RMSE is the root of the average of the squares of the differences between the predicted and the actual values
# see the documentation for more details: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator

# the higher the RMSE, the worse the model
# the lower the RMSE, the better the model
# the RMSE is a measure of the quality of the model
# the RMSE is scale-dependent
# the RMSE can be used to compare different models
# the RMSE can be used to compare different transformations of the same model
# the RMSE can be used to compare different models on different datasets
# the RMSE can be used to compare different models on the same dataset
# the RMSE can be used to compare different models on the same dataset with different target variables
# the RMSE can be used to compare different models on the same dataset with the same target variable
# the RMSE can be used to compare different models on the same dataset with the same target variable and different features
# the RMSE can be used to compare different models on the same dataset with the same target variable and the same features
# the RMSE can be used to compare different models on the same dataset with the same target variable and the same features and different hyperparameters
# the RMSE can be used to compare different models on the same dataset with the same target variable and the same features and the same hyperparameters
# the RMSE value of 0 means the model is perfect
# the RMSE value of 9 means the model is good?
avg_rmse = np.mean(cvModel.avgMetrics)
print("Average RMSE: ", avg_rmse)

Average RMSE:  8.486906382101072


In [41]:
# print all the parameters of the best model
print(cvModel.bestModel.extractParamMap())


{Param(parent='DecisionTreeRegressor_141f9f55ecef', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='DecisionTreeRegressor_141f9f55ecef', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='DecisionTreeRegressor_141f9f55ecef', name='featuresCol', doc='features column name.'): 'features', Param(parent='DecisionTreeRegressor_141f9f55ecef', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance', Para

In [42]:
# print the RMSE of all the models
for rmse in cvModel.avgMetrics:
    print(rmse)
    

9.393379984591112
9.390942372835946
9.403660773511568
9.401298199651212
9.393577950255418
9.01192778529562
9.038330752958633
8.998848717554736
8.992717896920754
8.988018135967867
8.786818109553082
8.82061828175019
8.735147100042486
8.719847202023796
8.708840129585116
8.608837784353293
8.610256844529527
8.538345281713893
8.525946908871491
8.51438484721434
8.473645443488289
8.438752166774313
8.36915541793392
8.356149163652079
8.34083699540852
8.3437519170105
8.287426251062765
8.226785648040522
8.213243254793085
8.205129889202498
8.24061459658503
8.144438432689366
8.08755612245877
8.08871903330523
8.080297843843265
8.135639834102198
8.01548224569062
7.942298051430379
7.9483595681559125
7.939560695428433
8.03494246532034
7.89398855988731
7.832045104057846
7.851093486999453
7.839129948047554


In [15]:
"""

an RMSE of 9.841781224229344 means that, on average, the model's predictions are about 9.84 units away from the actual values.

Whether this RMSE is considered good or bad depends on the context and the scale of your data. For example, if you are predicting stock prices that range from 100 to 200, an RMSE of 9.84 might be acceptable. However, if your stock prices range from 10 to 20, an RMSE of 9.84 would be quite high, indicating that your model's predictions are not very accurate.

It's also important to compare this RMSE to the performance of other models or benchmarks. If other models or approaches yield significantly lower RMSE values, then an RMSE of 9.84 might be considered suboptimal.

Considering your project involves predicting the exact prices of stocks, Brent oil, Ethereum, and gold, the evaluation of the RMSE value depends on the specific context:

Stock Prices: If the stock prices you're predicting have a wide range (e.g., $50 to $300), an RMSE of 9.84 might be considered reasonable. However, for lower-priced stocks or if you aim for high precision, this RMSE could be considered high.
Brent Oil Prices: The price range for Brent oil can vary significantly depending on market conditions. If the range is broad, an RMSE of 9.84 might be acceptable. However, for more stable periods with narrower price ranges, this value might be too high.
Ethereum Prices: Given the volatility and wide range of cryptocurrency prices, an RMSE of 9.84 could be considered reasonable, but this depends on the specific price range during your prediction period.
Gold Prices: Similar to Brent oil, the evaluation of the RMSE for gold prices depends on the price range during the period you're predicting. For a wide range, an RMSE of 9.84 might be acceptable, but for a narrower range, it could be considered high.
Overall, the acceptability of an RMSE of 9.84 in your project depends on the specific price ranges and volatility of the assets you're predicting. It's also important to compare this value to other models or benchmarks to evaluate its relative performance.


"""

"\n\nan RMSE of 9.841781224229344 means that, on average, the model's predictions are about 9.84 units away from the actual values.\n\nWhether this RMSE is considered good or bad depends on the context and the scale of your data. For example, if you are predicting stock prices that range from 100 to 200, an RMSE of 9.84 might be acceptable. However, if your stock prices range from 10 to 20, an RMSE of 9.84 would be quite high, indicating that your model's predictions are not very accurate.\n\nIt's also important to compare this RMSE to the performance of other models or benchmarks. If other models or approaches yield significantly lower RMSE values, then an RMSE of 9.84 might be considered suboptimal.\n\nConsidering your project involves predicting the exact prices of stocks, Brent oil, Ethereum, and gold, the evaluation of the RMSE value depends on the specific context:\n\nStock Prices: If the stock prices you're predicting have a wide range (e.g., $50 to $300), an RMSE of 9.84 might 

In [44]:
# print the feature importances

# The feature importances represent the importance of each feature in the decision tree model.

# The feature importances are calculated based on the information gain provided by each feature in splitting the data.

# The higher the feature importance, the more important the feature is in making predictions.



# get the feature importances
feature_importances = cvModel.bestModel.featureImportances

# create a DataFrame to display the feature importances
feature_importances_df = pd.DataFrame(list(zip(stocks_brent_gold_df_spark.columns[2:], feature_importances)),
                                      columns=["Feature", "Importance"])

# sort the DataFrame by feature importance in descending order
feature_importances_df = feature_importances_df.sort_values("Importance", ascending=False)

# display the feature importances
print(feature_importances_df)



         Feature  Importance
0  trading_name     0.449911
5            low    0.279164
3           open    0.123852
4           high    0.063026
1         Sector    0.042812
2           Date    0.041235


In [43]:
# print the feature importances
# Feature importances describe the relative importance of each feature in making accurate predictions.
# The higher the value, the more important the feature is.
# Feature importances can help you understand which features are most influential in your model's predictions.
# Feature importances can guide feature selection and model interpretation.

# get the feature importances
feature_importances = cvModel.bestModel.featureImportances

# create a pandas DataFrame to display the feature importances
feature_importances_df = pd.DataFrame(list(zip(["Stock_Price", "Brent_Price", "Gold_Price"], feature_importances)),
                                      columns=["Feature", "Importance"])

# display the feature importances
print(feature_importances_df)

       Feature  Importance
0  Stock_Price    0.449911
1  Brent_Price    0.042812
2   Gold_Price    0.041235


In [33]:
# print the hyperparameters of the best model
# The hyperparameters of the best model can provide insights into the configuration that yielded the best performance.
# Understanding the hyperparameters can help you tune future models and optimize their performance.
# The hyperparameters of the best model can guide further experimentation and refinement of the model.
print(cvModel.bestModel._java_obj.getMaxDepth())
print(cvModel.bestModel._java_obj.getMaxBins())

6
100


In [41]:
# define a time-series cross-validator
# TimeSeriesCrossValidator is a cross-validator for time series data.
# It works by splitting the dataset into training and test sets based on a time column.
# This allows you to evaluate the model's performance on unseen future data.
# TimeSeriesCrossValidator is useful for time series forecasting tasks where the order of data points matters.
# see the documentation for more details: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.TimeSeriesCrossValidator

# define the time-series cross-validator
# TimeSeriesCrossValidator requires an Estimator, a set of Estimator ParamMaps, an Evaluator, and a timeCol parameter.
# The timeCol parameter specifies the column that contains the time information for splitting the dataset.
# In this case, we use the "Date" column as the time column.
# see the documentation for more details: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.TimeSeriesCrossValidator
from pyspark.ml.tuning import TimeSeriesCrossValidator

# define the time-series cross-validator
tscv = TimeSeriesCrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, timeCol="Date", numFolds=5)

# fit the time-series cross-validator with the assembled data
tscvModel = tscv.fit(stocks_brent_gold_df_spark)

# get the average cross-validated RMSE
avg_rmse_tscv = np.mean(tscvModel.avgMetrics)
print("Average RMSE (Time Series Cross-Validation): ", avg_rmse_tscv)

# print all the parameters of the best model
print(tscvModel.bestModel.extractParamMap())

# print the RMSE of all the models
for rmse in tscvModel.avgMetrics:
    print(rmse)

# print the feature importances
feature_importances_tscv = tscvModel.bestModel.featureImportances
feature_importances_df_tscv = pd.DataFrame(list(zip(["Stock_Price", "Brent_Price", "Gold_Price"], feature_importances_tscv)),
                                          columns=["Feature", "Importance"])
print(feature_importances_df_tscv)

# print the hyperparameters of the best model
print(tscvModel.bestModel._java_obj.getMaxDepth())
print(tscvModel.bestModel._java_obj.getMaxBins())

ImportError: cannot import name 'TimeSeriesCrossValidator' from 'pyspark.ml.tuning' (/Users/bandaralghamdi/anaconda3/lib/python3.11/site-packages/pyspark/ml/tuning.py)

In [44]:
# Stop the Spark session
spark.stop()